In [1]:
!pip install sqlalchemy

In [2]:
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import time

# ---------------- LOGGING CONFIG ----------------
os.makedirs("logs", exist_ok=True)

logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

# ---------------- DB ENGINE ----------------
engine = create_engine("sqlite:///inventory.db")

# ---------------- CHUNKED INGEST FUNCTION ----------------
def ingest_db_chunked(csv_path, table_name, engine, chunksize=100_000):
    for chunk in pd.read_csv(csv_path, chunksize=chunksize):
        chunk.to_sql(
            table_name,
            con=engine,
            if_exists="append",
            index=False
        )

# ---------------- LOAD ALL CSVs ----------------
def load_raw_data():
    start = time.time()
    logging.info("Starting data ingestion")

    for file in os.listdir("data"):
        if file.endswith(".csv"):
            logging.info(f"Ingesting {file} into database")
            print(f"Ingesting {file}...")
            
            ingest_db_chunked(
                csv_path=f"data/{file}",
                table_name=file[:-4],
                engine=engine
            )

    end = time.time()
    total_time = (end - start) / 60

    logging.info("---------------- ingestion complete ----------------")
    logging.info(f"Total time taken: {total_time:.2f} minutes")

# ---------------- MAIN ----------------
if __name__ == "__main__":
    load_raw_data()


Ingesting begin_inventory.csv...
Ingesting end_inventory.csv...
Ingesting purchases.csv...
Ingesting purchase_prices.csv...
Ingesting sales.csv...
Ingesting vendor_invoice.csv...


In [3]:
import pandas as pd 
import sqlite3

In [4]:
conn = sqlite3.connect('inventory.db')

In [5]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type = 'table'",conn)
tables

,name
0,purchases
1,begin_inventory
2,end_inventory
3,purchase_prices
4,sales
5,vendor_invoice


In [6]:
pd.read_sql("select count(*) from purchases", conn)

,count(*)
0,7117422


In [7]:
for table in tables['name']:
    print('-'*50, f'{table}','-'*50)
    print('Count of records:', pd.read_sql(f"select count(*) as count from {table}", conn)['count'].values[0])
    display(pd.read_sql(f"select * from {table} limit 5", conn))

-------------------------------------------------- purchases --------------------------------------------------
Count of records: 7117422


,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,35.71,6,214.26,1
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
3,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
4,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,2024-01-09,2024-02-16,21.32,5,106.60,1


-------------------------------------------------- begin_inventory --------------------------------------------------
Count of records: 826116


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750mL,7,10.99,2024-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,6,36.99,2024-01-01
3,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,3,38.99,2024-01-01
4,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,6,34.99,2024-01-01


-------------------------------------------------- end_inventory --------------------------------------------------
Count of records: 897956


...

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


-------------------------------------------------- sales --------------------------------------------------
Count of records: 38476089


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546,JIM BEAM BRANDS COMPANY


-------------------------------------------------- vendor_invoice --------------------------------------------------
Count of records: 16629


,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61,None
3,480,BACARDI USA INC,2024-01-12,8106,2023-12-20,2024-02-05,10100,137483.78,2935.20,None
4,516,BANFI PRODUCTS CORP,2024-01-07,8170,2023-12-24,2024-02-12,1935,15527.25,429.20,None


In [8]:
purchases = pd.read_sql_query("select * from purchases where VendorNumber = 516" , conn)
purchases

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,30_CULCHETH_8678,30,8678,Banfi Chianti Clsc RSV,750mL,516,BANFI PRODUCTS CORP,8170,2023-12-24,2024-01-01,2024-01-07,2024-02-12,9.32,48,447.36,2
1,50_MOUNTMEND_6240,50,6240,Riunite Raspberry,1.5L,516,BANFI PRODUCTS CORP,8170,2023-12-24,2024-01-02,2024-01-07,2024-02-12,5.36,6,32.16,2
2,34_PITMERDEN_18027,34,18027,Banfi Rosa Regale,750mL,516,BANFI PRODUCTS CORP,8170,2023-12-24,2024-01-02,2024-01-07,2024-02-12,9.08,1,9.08,2
3,29_AYLESBURY_8678,29,8678,Banfi Chianti Clsc RSV,750mL,516,BANFI PRODUCTS CORP,8170,2023-12-24,2024-01-02,2024-01-07,2024-02-12,9.32,24,223.68,2
4,7_STANMORE_8678,7,8678,Banfi Chianti Clsc RSV,750mL,516,BANFI PRODUCTS CORP,8170,2023-12-24,2024-01-02,2024-01-07,2024-02-12,9.32,46,428.72,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70525,65_LUTON_18347,65,18347,Banfi Pnt Grigio San Angelo,750mL,516,BANFI PRODUCTS CORP,13570,2024-12-19,2024-12-28,2025-01-02,2025-02-11,8.58,23,197.34,2
70526,50_MOUNTMEND_16829,50,16829,C Y Toro Gran Reserva Cab Sv,750mL,516,BANFI PRODUCTS CORP,13570,2024-12-19,2024-12-30,2025-01-02,2025-02-11,9.52,6,57.12,2
70527,69_MOUNTMEND_8678,69,8678,Banfi Chianti Clsc RSV,750mL,516,BANFI PRODUCTS CORP,13570,2024-12-19,2024-12-30,2025-01-02,2025-02-11,9.32,24,223.68,2
70528,18_FURNESS_6859,18,6859,C Y Toro Frontera Chard,1.5L,516,BANFI PRODUCTS CORP,13570,2024-12-19,2024-12-29,2025-01-02,2025-02-11,4.69,18,84.42,2


In [9]:
purchase_prices = pd.read_sql_query("select * from purchase_prices where VendorNumber = 516" , conn)
purchase_prices

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,6234,C Y Toro Front Cab/Merlot,7.99,1500mL,1500,2,4.54,516,BANFI PRODUCTS CORP
1,6344,Walnut Crest Chard Central,11.99,1500mL,1500,2,6.62,516,BANFI PRODUCTS CORP
2,6429,C Y Toro Frontera Svgn Bl,8.99,1500mL,1500,2,5.80,516,BANFI PRODUCTS CORP
3,6478,C Y Toro Frontera Merlot,8.99,1500mL,1500,2,4.72,516,BANFI PRODUCTS CORP
4,6859,C Y Toro Frontera Chard,8.99,1500mL,1500,2,4.69,516,BANFI PRODUCTS CORP
...,...,...,...,...,...,...,...,...,...
442,25270,Cantine Maschio Sparkling Ro,3.79,187mL,187,2,2.46,516,BANFI PRODUCTS CORP
443,37924,Five Rivers Chard,12.99,750mL,750,2,8.60,516,BANFI PRODUCTS CORP
444,43819,Little Black Dress Chard,8.99,750mL,750,2,5.80,516,BANFI PRODUCTS CORP
445,6940,Bolla Merlot,9.99,750mL,750,2,6.57,516,BANFI PRODUCTS CORP


In [10]:
vendor_invoice = pd.read_sql_query("select * from vendor_invoice where VendorNumber = 516" , conn)
vendor_invoice

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,516,BANFI PRODUCTS CORP,2024-01-07,8170,2023-12-24,2024-02-12,1935,15527.25,429.20,None
1,516,BANFI PRODUCTS CORP,2024-01-14,8208,2023-12-27,2024-02-21,5458,41823.14,204.93,None
2,516,BANFI PRODUCTS CORP,2024-01-21,8339,2024-01-05,2024-02-20,4381,29909.78,140.58,None
3,516,BANFI PRODUCTS CORP,2024-02-01,8402,2024-01-10,2024-03-09,4252,30048.37,162.26,None
4,516,BANFI PRODUCTS CORP,2024-02-06,8570,2024-01-21,2024-03-20,5023,35384.60,162.77,None
...,...,...,...,...,...,...,...,...,...,...
160,516,BANFI PRODUCTS CORP,2024-12-07,13135,2024-11-20,2025-01-21,4409,33511.87,150.80,None
161,516,BANFI PRODUCTS CORP,2024-12-16,13239,2024-11-27,2025-01-14,3446,24145.77,127.97,None
162,516,BANFI PRODUCTS CORP,2024-12-23,13413,2024-12-08,2025-01-21,3867,31581.13,161.06,None
163,516,BANFI PRODUCTS CORP,2024-12-27,13499,2024-12-14,2025-02-09,3387,25651.30,128.26,None


In [11]:
sales = pd.read_sql_query("select * from sales where VendorNo = 516" , conn)
sales

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_10874,1,10874,Cantine Maschio Prosecco Brt,750mL,1,10.99,10.99,2024-01-07,750.0,2,0.11,516,BANFI PRODUCTS CORP
1,1_HARDERSFIELD_10874,1,10874,Cantine Maschio Prosecco Brt,750mL,1,10.99,10.99,2024-01-08,750.0,2,0.11,516,BANFI PRODUCTS CORP
2,1_HARDERSFIELD_10874,1,10874,Cantine Maschio Prosecco Brt,750mL,1,10.99,10.99,2024-01-15,750.0,2,0.11,516,BANFI PRODUCTS CORP
3,1_HARDERSFIELD_10874,1,10874,Cantine Maschio Prosecco Brt,750mL,14,153.86,10.99,2024-01-23,750.0,2,1.57,516,BANFI PRODUCTS CORP
4,1_HARDERSFIELD_10874,1,10874,Cantine Maschio Prosecco Brt,750mL,6,65.94,10.99,2024-01-29,750.0,2,0.68,516,BANFI PRODUCTS CORP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330562,9_BLACKPOOL_8790,9,8790,Bolla Valpolicella,1.5L,3,47.97,15.99,2024-12-18,1500.0,2,0.68,516,BANFI PRODUCTS CORP
330563,9_BLACKPOOL_8790,9,8790,Bolla Valpolicella,1.5L,1,15.99,15.99,2024-12-28,1500.0,2,0.22,516,BANFI PRODUCTS CORP
330564,9_BLACKPOOL_8790,9,8790,Bolla Valpolicella,1.5L,1,15.99,15.99,2024-12-30,1500.0,2,0.22,516,BANFI PRODUCTS CORP
330565,9_BLACKPOOL_8978,9,8978,Riunite Lambrusco,3L,2,33.98,16.99,2024-12-11,3000.0,2,0.90,516,BANFI PRODUCTS CORP


In [12]:
purchases.groupby(['Brand', 'PurchasePrice'])[['Quantity', 'Dollars']].sum()

,,Quantity,Dollars
Brand,PurchasePrice,,
199,3.87,36,139.32
5167,23.01,168,3865.68
6234,4.54,33828,153579.12
6240,5.36,3774,20228.64
6344,6.62,1905,12611.10
...,...,...,...
43820,5.88,4185,24607.80
43821,5.36,72,385.92
45524,5.80,2610,15138.00


In [13]:
purchase_prices

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,6234,C Y Toro Front Cab/Merlot,7.99,1500mL,1500,2,4.54,516,BANFI PRODUCTS CORP
1,6344,Walnut Crest Chard Central,11.99,1500mL,1500,2,6.62,516,BANFI PRODUCTS CORP
2,6429,C Y Toro Frontera Svgn Bl,8.99,1500mL,1500,2,5.80,516,BANFI PRODUCTS CORP
3,6478,C Y Toro Frontera Merlot,8.99,1500mL,1500,2,4.72,516,BANFI PRODUCTS CORP
4,6859,C Y Toro Frontera Chard,8.99,1500mL,1500,2,4.69,516,BANFI PRODUCTS CORP
...,...,...,...,...,...,...,...,...,...
442,25270,Cantine Maschio Sparkling Ro,3.79,187mL,187,2,2.46,516,BANFI PRODUCTS CORP
443,37924,Five Rivers Chard,12.99,750mL,750,2,8.60,516,BANFI PRODUCTS CORP
444,43819,Little Black Dress Chard,8.99,750mL,750,2,5.80,516,BANFI PRODUCTS CORP
445,6940,Bolla Merlot,9.99,750mL,750,2,6.57,516,BANFI PRODUCTS CORP


In [14]:
vendor_invoice['PONumber'].nunique()

55

In [15]:
vendor_invoice.columns

Index(['VendorNumber', 'VendorName', 'InvoiceDate', 'PONumber', 'PODate',
       'PayDate', 'Quantity', 'Dollars', 'Freight', 'Approval'],
      dtype='object')

In [16]:
sales.groupby('Brand')[['SalesDollars','SalesPrice','SalesQuantity']].sum()

,SalesDollars,SalesPrice,SalesQuantity
Brand,,,
199,239.52,119.76,48
5167,5953.44,4351.86,156
6234,278933.37,134455.44,30213
6240,35437.47,18530.76,4053
6344,21825.51,9731.79,2049
...,...,...,...
43820,40297.41,24435.60,4359
43821,491.52,392.61,48
45524,24749.31,12431.73,2469


• The purchase price column is derived from the purchase_prices table, which provides product-wise actual and purchase prices. The combination of vendor and brand is unique in this table.
• The vendor_invoice table aggregates data from the purchases table, summarizing quantity and dollar amounts, along with an additional column for freight. This table maintains uniqueness based on vendor and PO number.
• The sales table captures actual sales transactions, detailing the brands purchased by vendors, the quantity sold, the selling price, and the revenue earned.

As the data that we need for analysis is distributed in different tables, we need to create a summary table containing:

• purchase transactions made by vendors
• sales transaction data
• freight costs for each vendor
• actual product prices from vendors

In [17]:
freight_summary = pd.read_sql_query("""select VendorNumber, SUM(Freight) as FreightCost
From vendor_invoice
Group BY VendorNumber""", conn)

In [18]:
freight_summary

,VendorNumber,FreightCost
0,2,81.24
1,54,1.44
2,60,1102.56
3,105,187.17
4,200,18.57
...,...,...
121,98450,2568.06
122,99166,390.27
123,172662,535.02
124,173357,607.50


In [19]:
pd.read_sql_query("""SELECT
p.VendorNumber,
p.VendorName, 
p.Brand,
p.PurchasePrice,
pp.Volume,
pp.Price as ActualPrice,
SUM(p.Quantity) as TotalPurchaseQuantity,
SUM(p.Dollars) as TotalPurchaseDollars
FROM purchases p
JOIN purchase_prices pp
ON p.Brand = pp.Brand
Where p.PurchasePrice > 0
GROUP BY p.VendorNumber, p.VendorName, p.Brand
ORDER BY TotalPurchaseDollars""", conn)

,VendorNumber,VendorName,Brand,PurchasePrice,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars
0,7245,PROXIMO SPIRITS INC.,3065,0.71,50,0.99,9,6.39
1,3960,DIAGEO NORTH AMERICA INC,6127,1.47,200,1.99,9,13.23
2,3924,HEAVEN HILL DISTILLERIES,9123,0.74,50,0.99,18,13.32
3,8004,SAZERAC CO INC,5683,0.39,50,0.49,54,21.06
4,9815,WINE GROUP INC,8527,1.32,750,4.99,18,23.76
...,...,...,...,...,...,...,...,...
10687,3960,DIAGEO NORTH AMERICA INC,3545,21.89,1750,29.99,1242981,27208854.09
10688,3960,DIAGEO NORTH AMERICA INC,4261,16.17,1750,22.99,1815138,29350781.46
10689,17035,PERNOD RICARD USA,8068,18.24,1750,24.99,1686663,30764733.12
10690,4425,MARTIGNETTI COMPANIES,3405,23.19,1750,28.99,1476342,34236370.98


In [20]:
sales.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'SalesQuantity',
       'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume', 'Classification',
       'ExciseTax', 'VendorNo', 'VendorName'],
      dtype='object')

In [21]:
pd.read_sql_query(
    """SELECT
        VendorNo,
        Brand,
        SUM(SalesDollars) AS TotalSalesDollars,
        SUM(SalesPrice) AS TotalSalesPrice,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(ExciseTax) AS TotalExciseTax
        FROM sales
        GROUP BY VendorNo, Brand
        ORDER BY TotalSalesDollars""",conn
)


,VendorNo,Brand,TotalSalesDollars,TotalSalesPrice,TotalSalesQuantity,TotalExciseTax
0,8004,5287,2.94,2.94,6,0.30
1,9206,2773,2.97,2.97,3,0.15
2,3252,3933,5.94,2.97,6,0.30
3,3924,9123,5.94,2.97,6,0.30
4,10050,3623,5.94,5.94,6,0.30
...,...,...,...,...,...,...
11267,3960,3545,12669322.86,1637334.84,407514,748763.49
11268,3960,4261,13427918.64,1260150.03,601236,1104728.40
11269,17035,8068,13614361.80,1383420.45,561420,1031562.21
11270,4425,3405,14457220.47,1684537.11,480741,883315.98


In [22]:
import time

start = time.time()

vendor_sales_summary = pd.read_sql_query(
    """
    WITH FreightSummary AS (
        SELECT
            VendorNumber,
            SUM(Freight) AS FreightCost
        FROM vendor_invoice
        GROUP BY VendorNumber
    ),

    SalesSummary AS (
        SELECT
            VendorNo,
            Brand,
            SUM(SalesDollars) AS TotalSalesDollars,
            SUM(SalesPrice) AS TotalSalesPrice,
            SUM(SalesQuantity) AS TotalSalesQuantity,
            SUM(ExciseTax) AS TotalExciseTax
        FROM sales
        GROUP BY VendorNo, Brand
    ),

    PurchaseSummary AS (
        SELECT
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            p.PurchasePrice,
            pp.Volume,
            pp.Price AS ActualPrice,
            SUM(p.Quantity) AS TotalPurchaseQuantity,
            SUM(p.Dollars) AS TotalPurchaseDollars
        FROM purchases p
        JOIN purchase_prices pp
            ON p.Brand = pp.Brand
            AND p.VendorNumber = pp.VendorNumber
        WHERE p.PurchasePrice > 0
        GROUP BY
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            p.PurchasePrice,
            pp.Volume,
            pp.Price
    )

    SELECT
        ps.VendorNumber,
        ps.VendorName,
        ps.Brand,
        ps.PurchasePrice,
        ps.ActualPrice,
        ps.Volume,
        ps.TotalPurchaseQuantity,
        ps.TotalPurchaseDollars,
        ss.TotalSalesQuantity,
        ss.TotalSalesDollars,
        ss.TotalSalesPrice,
        ss.TotalExciseTax,
        fs.FreightCost
    FROM PurchaseSummary ps
    LEFT JOIN SalesSummary ss
        ON ps.VendorNumber = ss.VendorNo
        AND ps.Brand = ss.Brand
    LEFT JOIN FreightSummary fs
        ON ps.VendorNumber = fs.VendorNumber
    ORDER BY ps.TotalPurchaseDollars DESC
    """,
    conn
)

end = time.time()
print(f"Query executed in {(end - start):.2f} seconds")


Query executed in 295.75 seconds


In [23]:
vendor_sales_summary


,VendorNumber,VendorName,Brand,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost
0,1128,BROWN-FORMAN CORP,1233,26.27,36.99,1750,1305720,34301264.40,426147.0,15305758.53,2018457.93,782997.60,205805.04
1,4425,MARTIGNETTI COMPANIES,3405,23.19,28.99,1750,1476342,34236370.98,480741.0,14457220.47,1684537.11,883315.98,434787.72
2,17035,PERNOD RICARD USA,8068,18.24,24.99,1750,1686663,30764733.12,561420.0,13614361.80,1383420.45,1031562.21,371340.66
3,3960,DIAGEO NORTH AMERICA INC,4261,16.17,22.99,1750,1815138,29350781.46,601236.0,13427918.64,1260150.03,1104728.40,771096.21
4,3960,DIAGEO NORTH AMERICA INC,3545,21.89,29.99,1750,1242981,27208854.09,407514.0,12669322.86,1637334.84,748763.49,771096.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10643,9815,WINE GROUP INC,8527,1.32,4.99,750,18,23.76,15.0,47.85,32.88,1.65,81301.23
10644,8004,SAZERAC CO INC,5683,0.39,0.49,50,54,21.06,402.0,196.98,4.41,21.12,150880.86
10645,3924,HEAVEN HILL DISTILLERIES,9123,0.74,0.99,50,18,13.32,6.0,5.94,2.97,0.30,42209.61
10646,3960,DIAGEO NORTH AMERICA INC,6127,1.47,1.99,200,9,13.23,216.0,429.84,232.83,45.36,771096.21


This query generates a vendor-wise sales and purchase summary, which is valuable for:

Performance Optimization:

• The query involves heavy joins and aggregations on large datasets like sales and purchases.
• Storing the pre-aggregated results avoids repeated expensive computations.
• Helps in analyzing sales, purchases, and pricing for different vendors and brands.
• Future Benefits of Storing this data for faster Dashboard & Reporting.
• Instead of running expensive queries each time, dashboards can fetch data quickly from vendor_sales_summary.

In [24]:
vendor_sales_summary.dtypes

VendorNumber               int64
VendorName                object
Brand                      int64
PurchasePrice            float64
ActualPrice              float64
Volume                    object
TotalPurchaseQuantity      int64
TotalPurchaseDollars     float64
TotalSalesQuantity       float64
TotalSalesDollars        float64
TotalSalesPrice          float64
TotalExciseTax           float64
FreightCost              float64
dtype: object

In [25]:
vendor_sales_summary.isnull().sum()

VendorNumber               0
VendorName                 0
Brand                      0
PurchasePrice              0
ActualPrice                0
Volume                     0
TotalPurchaseQuantity      0
TotalPurchaseDollars       0
TotalSalesQuantity       178
TotalSalesDollars        178
TotalSalesPrice          178
TotalExciseTax           178
FreightCost                0
dtype: int64

In [26]:
vendor_sales_summary['VendorName'].unique()

array(['BROWN-FORMAN CORP          ', 'MARTIGNETTI COMPANIES',
       'PERNOD RICARD USA          ', 'DIAGEO NORTH AMERICA INC   ',
       'BACARDI USA INC            ', 'JIM BEAM BRANDS COMPANY    ',
       'MAJESTIC FINE WINES        ', 'ULTRA BEVERAGE COMPANY LLP ',
       'STOLI GROUP,(USA) LLC      ', 'PROXIMO SPIRITS INC.       ',
       'MOET HENNESSY USA INC      ', 'CAMPARI AMERICA            ',
       'SAZERAC CO INC             ', 'CONSTELLATION BRANDS INC   ',
       'M S WALKER INC             ', 'SAZERAC NORTH AMERICA INC. ',
       'PALM BAY INTERNATIONAL INC ', 'REMY COINTREAU USA INC     ',
       'SIDNEY FRANK IMPORTING CO  ', 'E & J GALLO WINERY         ',
       'WILLIAM GRANT & SONS INC   ', 'HEAVEN HILL DISTILLERIES   ',
       'DISARONNO INTERNATIONAL LLC', 'EDRINGTON AMERICAS         ',
       'CASTLE BRANDS CORP.        ', 'SOUTHERN WINE & SPIRITS NE ',
       'STE MICHELLE WINE ESTATES  ', 'TRINCHERO FAMILY ESTATES   ',
       'MHW LTD                    ', 'W

In [27]:
vendor_sales_summary = pd.read_sql_query(
    """
    WITH PurchaseSummary AS (
        SELECT
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            pp.Description,
            p.PurchasePrice,
            pp.Volume,
            pp.Price AS ActualPrice,
            SUM(p.Quantity) AS TotalPurchaseQuantity,
            SUM(p.Dollars) AS TotalPurchaseDollars
        FROM purchases p
        JOIN purchase_prices pp
            ON p.Brand = pp.Brand
            AND p.VendorNumber = pp.VendorNumber
        WHERE p.PurchasePrice > 0
        GROUP BY
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            pp.Description,
            p.PurchasePrice,
            pp.Volume,
            pp.Price
    )
    SELECT * FROM PurchaseSummary""",
    conn
)


In [28]:
vendor_sales_summary['Description'].unique()


array(['Ch Lilian 09 Ladouys St Este', 'Flavor Essence Variety 5 Pak',
       'Ethyl Alcohol 200 Proof', ..., 'Art in the Age Chicory Root',
       'Chicory Root Vodka', 'White Mountain Vodka'], dtype=object)

In [29]:
vendor_sales_summary['VendorName'].unique()


array(['IRA GOLDMAN AND WILLIAMS, LLP          ',
       'AAPER ALCOHOL & CHEMICAL CO', 'ADAMBA IMPORTS INTL INC    ',
       'ALTAMAR BRANDS LLC         ', 'AMERICAN SPIRITS EXCHANGE  ',
       'APPOLO VINEYARDS LLC       ', 'ATLANTIC IMPORTING COMPANY ',
       'BACARDI USA INC            ', 'BANFI PRODUCTS CORP        ',
       'STATE WINE & SPIRITS   ', 'SAZERAC NORTH AMERICA INC. ',
       'BRONCO WINE COMPANY        ', 'BROWN-FORMAN CORP          ',
       'BULLY BOY DISTILLERS       ', 'BLACK ROCK SPIRITS LLC     ',
       'CALEDONIA SPIRITS INC      ', 'CONSTELLATION BRANDS INC   ',
       'CAPSTONE INTERNATIONAL     ', 'CASTLE BRANDS CORP.        ',
       'VINEYARD BRANDS INC        ', 'VINEYARD BRANDS LLC        ',
       'DIAGEO CHATEAU ESTATE WINES', 'VRANKEN AMERICA            ',
       'Circa Wines                ', 'FABRIZIA SPIRITS LLC       ',
       'ALISA CARR BEVERAGES           ', 'SOUTHERN GLAZERS W&S OF NE ',
       'SOUTHERN WINE & SPIRITS NE ', 'DELICATO VINEY

In [30]:
vendor_sales_summary['Volume'] = vendor_sales_summary['Volume'].astype('float64')



In [31]:
vendor_sales_summary.fillna(0, inplace = True)

In [32]:
vendor_sales_summary['VendorName'] = vendor_sales_summary['VendorName'].str.strip()



In [33]:
vendor_sales_summary.columns

Index(['VendorNumber', 'VendorName', 'Brand', 'Description', 'PurchasePrice',
       'Volume', 'ActualPrice', 'TotalPurchaseQuantity',
       'TotalPurchaseDollars'],
      dtype='object')

In [34]:
print(list(vendor_sales_summary.columns))


['VendorNumber', 'VendorName', 'Brand', 'Description', 'PurchasePrice', 'Volume', 'ActualPrice', 'TotalPurchaseQuantity', 'TotalPurchaseDollars']


In [35]:
vendor_sales_summary = pd.read_sql_query(
    """
    WITH SalesSummary AS (
        SELECT
            VendorNo,
            Brand,
            SUM(SalesDollars) AS TotalSalesDollars,
            SUM(SalesQuantity) AS TotalSalesQuantity,
            SUM(ExciseTax) AS TotalExciseTax
        FROM sales
        GROUP BY VendorNo, Brand
    )
    SELECT * FROM SalesSummary""",
    conn
)


In [36]:
vendor_sales_summary = pd.read_sql_query(
    """
    WITH SalesSummary AS (
        SELECT
            VendorNo,
            Brand,
            SUM(SalesDollars) AS TotalSalesDollars,
            SUM(SalesQuantity) AS TotalSalesQuantity,
            SUM(ExciseTax) AS TotalExciseTax
        FROM sales
        GROUP BY VendorNo, Brand
    ),
    PurchaseSummary AS (
        SELECT
            VendorNumber,
            Brand,
            SUM(Dollars) AS TotalPurchaseDollars
        FROM purchases
        GROUP BY VendorNumber, Brand
    )
    SELECT
        ss.VendorNo,
        ss.Brand,
        ss.TotalSalesDollars,
        ss.TotalSalesQuantity,
        ss.TotalExciseTax,
        ps.TotalPurchaseDollars
    FROM SalesSummary ss
    LEFT JOIN PurchaseSummary ps
        ON ss.VendorNo = ps.VendorNumber
        AND ss.Brand = ps.Brand
    """,
    conn
)


In [37]:
vendor_sales_summary['GrossProfit'] = (
    vendor_sales_summary['TotalSalesDollars']
    - vendor_sales_summary['TotalPurchaseDollars']
)


In [38]:
vendor_sales_summary['GrossProfit'].min()

-156008.34

In [39]:
vendor_sales_summary['ProfitMargin'] =(vendor_sales_summary['GrossProfit'] / vendor_sales_summary['TotalSalesDollars'])*100

In [40]:
pd.read_sql_query("""SELECT
p.VendorNumber,
p.VendorName, 
p.Brand,
p.PurchasePrice,
pp.Volume,
pp.Price as ActualPrice,
SUM(p.Quantity) as TotalPurchaseQuantity,
SUM(p.Dollars) as TotalPurchaseDollars
FROM purchases p
JOIN purchase_prices pp
ON p.Brand = pp.Brand
Where p.PurchasePrice > 0
GROUP BY p.VendorNumber, p.VendorName, p.Brand
ORDER BY TotalPurchaseDollars""", conn)

,VendorNumber,VendorName,Brand,PurchasePrice,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars
0,7245,PROXIMO SPIRITS INC.,3065,0.71,50,0.99,9,6.39
1,3960,DIAGEO NORTH AMERICA INC,6127,1.47,200,1.99,9,13.23
2,3924,HEAVEN HILL DISTILLERIES,9123,0.74,50,0.99,18,13.32
3,8004,SAZERAC CO INC,5683,0.39,50,0.49,54,21.06
4,9815,WINE GROUP INC,8527,1.32,750,4.99,18,23.76
...,...,...,...,...,...,...,...,...
10687,3960,DIAGEO NORTH AMERICA INC,3545,21.89,1750,29.99,1242981,27208854.09
10688,3960,DIAGEO NORTH AMERICA INC,4261,16.17,1750,22.99,1815138,29350781.46
10689,17035,PERNOD RICARD USA,8068,18.24,1750,24.99,1686663,30764733.12
10690,4425,MARTIGNETTI COMPANIES,3405,23.19,1750,28.99,1476342,34236370.98


In [41]:
vendor_sales_summary['StockTurnover'] = vendor_sales_summary['TotalSalesQunatity'] / vendor_sales_summary['TotalPurchaseQuantity']

KeyError: 'TotalSalesQunatity'

In [ ]:
vendor_sales_summary['SalesPurchaseRatio'] = vendor_sales_summary['TotalSalesDollars'] / vendor_sales_summary['TotalPurchaseDollars']

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute("""CREATE TABLE vendor_sales_summary (
    VendorNumber INT,
    VendorName VARCHAR(100),
    Brand INT,
    Description VARCHAR(100),
    PurchasePrice DECIMAL(10,2),
    ActualPrice DECIMAL(10,2),
    Volume INT,
    TotalPurchaseQuantity INT,
    TotalPurchaseDollars DECIMAL(15,2),
    TotalSalesQuantity INT,
    TotalSalesDollars DECIMAL(15,2),
    TotalSalesPrice DECIMAL(15,2),
    TotalExciseTax DECIMAL(15,2),
    FreightCost DECIMAL(15,2),
    GrossProfit DECIMAL(15,2),
    ProfitMargin DECIMAL(15,2),
    StockTurnover DECIMAL(15,2),
    SalesToPurchaseRatio DECIMAL(15,2),
    PRIMARY KEY (VendorNumber, Brand)
);
""")


In [42]:
vendor_sales_summary.to_sql('vendor_sales_summary', conn, if_exists = 'replace', index = False)

11272

In [43]:
pd.read_sql_query("select * from vendor_sales_summary",conn)

,VendorNo,Brand,TotalSalesDollars,TotalSalesQuantity,TotalExciseTax,TotalPurchaseDollars,GrossProfit,ProfitMargin
0,2,90085,1997.46,54,6.00,572.64,1424.82,71.331591
1,2,90609,1799.28,72,1.56,16320.00,-14520.72,-807.029478
2,60,771,2113.59,141,111.03,1338.48,775.11,36.672675
3,60,3979,200615.07,11793,21672.18,228772.47,-28157.40,-14.035536
4,105,2529,1079.64,36,28.32,837.00,242.64,22.474158
...,...,...,...,...,...,...,...,...
11267,173357,2804,18895.80,420,330.99,20248.20,-1352.40,-7.157146
11268,173357,3666,26989.20,1080,423.57,29312.40,-2323.20,-8.607888
11269,173357,3848,557.82,18,14.13,1957.20,-1399.38,-250.865871
11270,173357,3909,73620.54,2946,2321.61,71591.52,2029.02,2.756052


In [49]:
import sqlite3
import pandas as pd
import logging
from ingestion_db import ingest_db_chunked  # make sure this module exists

# ------------------ Logging Setup ------------------
logging.basicConfig(
    filename="logs/get_vendor_summary.logs",  # fixed typo 'filenmae'
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

# ------------------ Function: Create Vendor Summary ------------------
def create_vendor_summary(conn):
    """This function merges different tables to get the overall vendor summary
    and adds relevant columns for analysis."""
    
    vendor_sales_summary = pd.read_sql_query(
        """
WITH FreightSummary AS (
    SELECT
        VendorNumber,
        SUM(Freight) AS FreightCost
    FROM vendor_invoice
    GROUP BY VendorNumber
),

PurchaseSummary AS (
    SELECT
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        p.PurchasePrice,
        pp.Price AS ActualPrice,
        pp.Volume,
        SUM(p.Quantity) AS TotalPurchaseQuantity,
        SUM(p.Dollars) AS TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
        ON p.Brand = pp.Brand
        AND p.VendorNumber = pp.VendorNumber
    WHERE p.PurchasePrice > 0
    GROUP BY
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        p.PurchasePrice,
        pp.Price,
        pp.Volume
),

SalesSummary AS (
    SELECT
        VendorNo,
        Brand,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(SalesDollars) AS TotalSalesDollars,
        SUM(SalesPrice) AS TotalSalesPrice,
        SUM(ExciseTax) AS TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand
)

SELECT
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalSalesPrice,
    ss.TotalExciseTax,
    fs.FreightCost
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss.VendorNo
    AND ps.Brand = ss.Brand
LEFT JOIN FreightSummary fs
    ON ps.VendorNumber = fs.VendorNumber
ORDER BY ps.TotalPurchaseDollars DESC
        """,
        conn
    )
    
    return vendor_sales_summary

# ------------------ Function: Clean Data ------------------
def clean_data(df):
    """This function cleans the data and creates additional analysis columns."""
    
    # Convert 'Volume' to float
    df['Volume'] = df['Volume'].astype('float')
    
    # Fill missing values with 0
    df.fillna(0, inplace=True)
    
    # Remove spaces from categorical columns
    df['VendorName'] = df['VendorName'].str.strip()
    df['Description'] = df['Description'].str.strip()
    
    # Create new analysis columns
    df['GrossProfit'] = df['TotalSalesDollars'] - df['TotalPurchaseDollars']
    df['ProfitMargin'] = (df['GrossProfit'] / df['TotalSalesDollars']) * 100
    df['StockTurnover'] = df['TotalSalesQuantity'] / df['TotalPurchaseQuantity']
    df['SalesToPurchaseRatio'] = df['TotalSalesDollars'] / df['TotalPurchaseDollars']
    
    return df

# ------------------ Main Script ------------------
if __name__ == '__main__':
    
    conn = sqlite3.connect('inventory.db')
    
    logging.info('Creating Vendor Summary Table...')
    summary_df = create_vendor_summary(conn)
    logging.info(f'Sample Data:\n{summary_df.head()}')
    
    logging.info('Cleaning Data...')
    clean_df = clean_data(summary_df)
    logging.info(f'Cleaned Data Sample:\n{clean_df.head()}')
    
    logging.info('Ingesting data into database...')
    clean_df.to_sql(
        'vendor_sales_summary',
        conn,
        if_exists='replace',
        index=False
    )
    
    logging.info('Process Completed Successfully.')
    conn.close()
